<a href="https://colab.research.google.com/github/Gutobastos/imersao-iA-3-Alura/blob/main/projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai
%pip -q install google-adk

In [2]:
import os

# E necessário inserir a chave na variável do seu ambiente com sua GOOGLE_API_KEY
google_api_key = os.environ.get('GOOGLE_API_KEY')
if google_api_key:
    print("Chave da API do Google encontrada nas variáveis de ambiente do seu PC.\n")
else:
    print("Chave da API do Google não encontrada nas variáveis de ambiente e tentaremos pelo GOOGLE COLAB.\n")
    from google.colab import userdata
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

Chave da API do Google encontrada nas variáveis de ambiente do seu PC.



In [3]:
from google import genai
client = genai.Client()
modelo = "gemini-2.0-flash"

In [4]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings

warnings.filterwarnings("ignore")

In [5]:
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [6]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [7]:
##########################################
# --- Agente 1: BUSCADOR --- #
##########################################
def agente_buscador(topico, data_de_hoje):
  buscador = Agent(
      name = "agente_buscador",
      model = "gemini-2.0-flash",
      description = "Agente que busca informações no Google",
      tools = [google_search],
      instruction = """
      Você é um assistente de pesquisa de móveis ou produtos e deve usar a ferramenta de 
      busca do Google (google_search), procure pelo móvel e que tenha preço e sejam o mais barato,
      faça uma planilha de preços no final. 
      Que tenha garantia e boa referência de reputação e avaliação a partir da data de hoje.
      Somente guarde resultados que possuem preços.
      """
  )
  entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

  moveis = call_agent(buscador, entrada_do_agente_buscador)
  return moveis

In [8]:
################################################
# --- Agente 2: ORÇAMENTO --- #
################################################
def agente_cotador(topico, moveis_buscados):
    planejador = Agent(
        name="agente_cotacao",
        model="gemini-2.0-flash",
        description="Agente cotação de preços",
        tools=[google_search],
        instruction="""
        Você é um filtro de pesquisa, especialista em encontrar o móvel ou produtos com melhor custo 
        benefícioa partir da data de hoje. Você deve usar a feramenta de busca do Google (google_search) e dar 
        prioridade preço mais baixo.
        Escolha de 1 até 5 móveis ou produtos que tenham preços e com formas de pagamentos se houver.
        """
    )
    entrada_do_agente_cotador = f"Tópico:{topico}\nLançamentos buscados: {moveis_buscados}"

    moveis_do_post = call_agent(planejador, entrada_do_agente_cotador)
    return moveis_do_post

In [9]:
######################################
# --- Agente 3: AVALIADOR --- #
######################################
def agente_avaliador(topico, avaliacao_de_post):
    redator = Agent(
        name="agente_avaliador",
        model="gemini-2.5-pro-preview-03-25",
        description="Agente avaliador de posts",
        instruction="""
            Você é um avaliador de controle de qualidade, você deve priorizar 
            as médias de avaliações, você deve usar a feramenta de busca do Google (google_search).
            Guarde os preços e inclua na tabela.
            Escolha até 5 produtos que tenha a melhor média de avaliação e faça a planilha.
            """
    )
    entrada_do_agente_avaliador = f"Tópico: {topico}\nPlano de post: {avaliacao_de_post}"

    rascunho = call_agent(redator, entrada_do_agente_avaliador)
    return rascunho

In [10]:
##########################################
# --- Agente 4: RESULTADO --- #
##########################################
def agente_retorno(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-pro-preview-03-25",
        description="Agente revisor de post para redes sociais.",
        instruction="""
            Você é um vendedor e dará informação da mais completa sobre o produto, 
            você é especializado vendas varejista, com foco no melhor móvel, com melhor custo, benefício e com menor prazo de entrega se tiver.
            Por ter um público, entre 20 e 50 anos, use um tom de escrita adequado, respeitoso e confiante.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho avaliado, se estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            Mostre uma tabelha com apenas 5 ítens e que tenham preço em ordem crescente, com o nome das empresas e suas avaliações positivas.
            Faça hiperlinks de cada produto da pesquisa inserindo na planilha.
            """
    )
    entrada_do_agente_retorno = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"

    texto_final = call_agent(revisor, entrada_do_agente_retorno)
    return texto_final

In [12]:
data_de_hoje = date.today().strftime("%d/%m/%Y")
loop = "play"
print("📜 Iniciando o Sistema de Pesquisa de Preços de móveis ou produtos com avaliação de qualidade com 4 Agentes IA do Google 📜\n")
while loop != "sair" or topico != "SAIR" or topico != "Sair":
  topico = input("🔍 Por favor, digite o tipo do Móvel ou Produto desejado sobre o qual você quer encontrar ou comprar 🔍: ")
  loop = topico
  if topico == "sair" or topico == "SAIR" or topico == "Sair":
    print("O programa foi encerrado com sucesso.")
    break
  if not topico:
    print("🤔 Você não digitou um tipo de móvel ou produto. Por favor, tente novamente ou digite SAIR para finalizar o programa. 🤔\n")
  else :
    print(f"💲 Pronto, vamos buscar pelo seu móvel ou produto e exibir o orçamento 💲: {topico}" )
    moveis_buscados = agente_buscador(topico, data_de_hoje)
    cotador_de_custos = agente_buscador (topico, moveis_buscados)
    avaliador_de_empresas = agente_buscador (topico, cotador_de_custos)
    resultado_do_post = agente_buscador (topico, avaliador_de_empresas)
    print("\n----------------- RESULTADO DOS AGENTES -----------------\n")
    display(to_markdown(resultado_do_post))
    print("----------------------------------------------------------------")

📜 Iniciando o Sistema de Pesquisa de Preços de móveis ou produtos com avaliação de qualidade com 4 Agentes IA do Google 📜

O programa foi encerrado com sucesso.
